# 비지도학습 군집화 테스트

###  "사용 데이터 : 장르별 베스트셀러 리뷰 모음"

### 조건
    * 각각의 ISBN끼리 리뷰를 합쳐서 테스트
    * 형용사 : 길이가 1이상
    * topics : 5
    * 테스트 : 15

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

%matplotlib inline

import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import urllib

In [2]:
import konlpy
from konlpy.tag import Okt

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


# 데이터 전처리

## 도서 리뷰 추가

In [4]:
import glob
import pandas as pd
from pandas.core.dtypes.missing import notnull
import nltk
from konlpy.tag import Okt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
import re
from gensim.models import Word2Vec
from pandas import DataFrame
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## <전처리>

In [5]:
def cleansing(text):
    # text = str(text)
    pattern = "([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)"  # e-mail 주소 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+"  # url 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "([^ㄷㅋㅎㅜㅠ가-힣a-zA-Z0-9])+"  # 한글 자음, 모음 제거 - without 'ㄷ,ㅋ,ㅎ,ㅜ,ㅠ'
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "<[^>]*>"  # html tag 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = "[\r|\n]"  # \r, \n 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    pattern = re.compile(r"\s+")  # 이중 space 제거
    text = re.sub(pattern=pattern, repl=" ", string=text)

    return text

### ISBN을 기준으로 리뷰를 모은 뒤 재조합

## <토큰화>

#### 분류 테스트 

In [6]:
okt = Okt()

----

### 기존 데이터 가져오기

In [7]:
# 불러오기
import pickle

book_morphs_with_isbn_pos_non_stopword = []
with open('data/detail_bestbook_unit_isbn_with_isbn_pos_non_stopword.txt', 'rb') as lf:
    book_morphs_with_isbn_pos_non_stopword = pickle.load(lf)
    
print(book_morphs_with_isbn_pos_non_stopword[0])

[9788959752195, [('내', 'Noun'), ('가제', 'Noun'), ('일', 'Noun'), ('좋아하다', 'Adjective'), ('책', 'Noun'), ('이', 'Noun'), ('당', 'Noun'), ('6', 'Number')]]


In [8]:
print(len(book_morphs_with_isbn_pos_non_stopword))

print(book_morphs_with_isbn_pos_non_stopword[10])

42559
[9788974825515, [('아이', 'Noun'), ('에게', 'Josa'), ('중국어', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('자신감', 'Noun'), ('을', 'Josa'), ('실어', 'Noun'), ('줄', 'Noun'), ('수', 'Noun'), ('있다', 'Adjective'), ('책', 'Noun'), ('인', 'Noun'), ('것', 'Noun'), ('같다', 'Adjective'), ('열심히', 'Adverb'), ('공부', 'Noun'), ('하다', 'Verb'), ('자격증', 'Noun'), ('을', 'Josa'), ('따다', 'Verb'), ('중국어', 'Noun'), ('와', 'Verb'), ('더', 'Noun'), ('가깝다', 'Adjective'), ('지다', 'Noun'), ('하다', 'Verb')]]


In [9]:
print(book_morphs_with_isbn_pos_non_stopword[0])

[9788959752195, [('내', 'Noun'), ('가제', 'Noun'), ('일', 'Noun'), ('좋아하다', 'Adjective'), ('책', 'Noun'), ('이', 'Noun'), ('당', 'Noun'), ('6', 'Number')]]


In [10]:
#  도서 불용어를 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','것','라고','에게','라면','을','이라',
             '라니','있다','아','랑','쯤된','에서','에선','어','이지만','으로나','때','때는','때라면','때라서','라','이다','있','죠','고','니','로','있','같',
             '어서','어요','는데','습니다','면서','많이','마','더','그렇다','의','당','좀','책','안','볼','게','안','정말','듯','이제야','여','요','게다가','같다',
            '임','로서','이제','만','인','붙이','그','저','수','가제','부터','닷','저희','적','알','쉬','못','꼭','살','제','권','제','분','나','내', '진작', '전', '뿐'
            , '대한','대해']

In [17]:
######### 태깅한 데이터 중에서 형용사(1이상) + 불용어 처리

book_train_adj = []
# book_train_na_with_isbn = []
for (isbn, test) in book_morphs_with_isbn_pos_non_stopword :
    tts = [(word,pos) for (word,pos) in test if not word in stopwords] #불용어 처리

#     tt = [(word,pos) for (word,pos) in tts if pos in 'Noun'] # 태깅데이터 포함
#     na = [word for (word,pos) in tts if pos in 'Noun'] # 명사만 저장
    na = []
    for (word,pos) in tts : #형용사 : 길이가 1이상
        if pos == 'Adjective' and len(word)>1 :
#             tt.append((word,pos))
            na.append(word)
    if na : 
        book_train_adj.append(na)
#         book_na_tag.append(tt)
#         book_train_na_with_isbn.append([isbn,na])

print('**********************END***************************')

**********************END***************************


In [18]:
for i in range(0,5) :
#     print(book_morphs_with_isbn_pos_non_stopword[i])
    print(book_train_adj[i])

['좋아하다']
['다르다', '좋다', '유쾌하다', '유치하다']
['귀엽다', '기쁘다', '좋다', '없다', '아쉽다', '아니다', '알차다']
['유익하다', '어렵다', '자세하다', '좋다', '훌륭하다', '없다']
['유용하다', '많다', '좋다']


----

# 정수 인코딩과 단어 집합 만들기

(인코딩값, 빈도수)

#### 데이터 셋
* 형용사 길이 > 1

In [20]:
from gensim import corpora


dictionary = corpora.Dictionary(book_train_adj)
corpus = [dictionary.doc2bow(text) for text in book_train_adj]

In [21]:
print(len(book_train_adj))
print(len(dictionary))
print(len(corpus))

39125
1606
39125


In [27]:
print(corpus[24983])

[(13, 1), (49, 1)]


In [28]:
print(dictionary[13])
print(dictionary[49])

자세하다
재미있다


# LDA 모델 훈련시키기


passes는 알고리즘의 동작 횟수

In [29]:
import gensim

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.127*"좋다" + 0.062*"아름답다" + 0.058*"따뜻하다" + 0.051*"예쁘다"')
(1, '0.185*"좋다" + 0.103*"많다" + 0.063*"어렵다" + 0.042*"아니다"')
(2, '0.449*"좋다" + 0.139*"좋아하다" + 0.038*"괜찮다" + 0.035*"없다"')
(3, '0.103*"없다" + 0.090*"아니다" + 0.039*"어떻다" + 0.032*"많다"')
(4, '0.180*"재미있다" + 0.140*"재밌다" + 0.070*"흥미롭다" + 0.054*"궁금하다"')


## 시각화


In [30]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [20]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.2716429), (1, 0.0673414), (2, 0.06820571), (3, 0.52559185), (4, 0.067218125)]
1 번째 문서의 topic 비율은 [(2, 0.290811), (3, 0.680233)]
2 번째 문서의 topic 비율은 [(3, 0.9609487)]
3 번째 문서의 topic 비율은 [(0, 0.028732767), (1, 0.029542703), (2, 0.02868899), (3, 0.88404244), (4, 0.0289931)]
4 번째 문서의 topic 비율은 [(0, 0.14995998), (3, 0.06903604), (4, 0.77366215)]
d:\project02\field-team4_Test\machine_learning\web_scraping\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
print(len(ldamodel[corpus]))

42486
d:\project02\field-team4_Test\machine_learning\web_scraping\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
